In [ ]:
%flink_sql_replace -s '{{{{.Values.flink.ngsildUpdateWindow|squote}}}}' -r "'0.001'"

In [ ]:
%%flink_gateway_sql_prepare
SET 'pipeline.name' = 'NGSI-LD insert job';

In [ ]:
%%flink_gateway_sql_query
INSERT INTO ngsild_updates
SELECT 
    'update', 
    true, 
    true, 
    '[{"id": "' || id || '",' || LISTAGG(attributeagg) || '}]'
FROM (
    SELECT  
        window_start, 
        window_end, 
        entityId AS id, 
        '"' || name || '":[' || LISTAGG(indexagg) || ']' AS attributeagg
    FROM (
        SELECT 
            window_start, 
            window_end, 
            entityId, 
            name,
            /* Construct the JSON string for the latest value */
            '{"observedAt": "' || DATE_FORMAT(ts, 'yyyy-MM-dd''T''HH:mm:ss.') || CAST(EXTRACT(MILLISECOND FROM ts) AS STRING) || 'Z", "type": "'
            || type
            || '", "datasetId": "'
            || IF(datasetId IS NOT NULL, datasetId, '@none')
            || IF(type = 'https://uri.etsi.org/ngsi-ld/Relationship', '", "object": "', '", "value": ')
            || IF(type = 'https://uri.etsi.org/ngsi-ld/Relationship', attributeValue || '"',
                IF(
                    (nodeType <> '@json' AND nodeType <> '@id') OR nodeType IS NULL,
                    '"' || attributeValue || '"',
                    IF(nodeType = '@id', '{"@id": "' || attributeValue || '"}',
                        IF(nodeType = '@json', attributeValue, 'null' /*should not happen */)
                    )
                )
            )
            || '}' AS indexagg
        FROM (
            SELECT *, 
                   ROW_NUMBER() OVER (
                       PARTITION BY entityId, name, datasetId, window_start, window_end
                       ORDER BY ts DESC
                   ) AS row_num
            FROM TABLE(
                TUMBLE(TABLE attributes, DESCRIPTOR(ts), INTERVAL {{.Values.flink.ngsildUpdateWindow|squote}} second)
            )
            WHERE entityId IS NOT NULL 
                AND (deleted IS NULL OR deleted = false) 
                AND (synced IS NULL OR synced = false)
        ) AS ranked_data
        WHERE row_num = 1 -- Select only the most recent value for each group
    ) AS filtered_data
    GROUP BY window_start, window_end, entityId, name
) AS aggregated_data
GROUP BY window_start, window_end, id;



In [ ]:
%%flink_gateway_sql_prepare
SET 'pipeline.name' = 'Attribute insert job';

In [ ]:
%flink_sql_replace -s '{{{{ .Values.flink.attributeInsertWindow |squote }}}}' -r "'0.001'"

In [ ]:
%%flink_gateway_sql_query
           /* forward only non-retracted attribute changes
            This is part of the core service */
            insert into attributes_writeback
            select id, parentId, entityId, name, nodeType, valueType, `type`, `attributeValue`, `datasetId`, `unitCode`, `deleted`, `synced` from
            (select id, last_value(parentId) as parentId, last_value(entityId) as entityId, last_value(name) as name, last_value(nodeType) as nodeType,
                last_value(valueType) as valueType , last_value(`type`) as `type`,
            last_value(`attributeValue`) as `attributeValue`,
            `datasetId` as `datasetId`, last_value(`unitCode`) as `unitCode`, last_value(`deleted`) as `deleted`, last_value(`synced`) as `synced`,
              TUMBLE_START(ts, INTERVAL {{ .Values.flink.attributeInsertWindow |squote }} SECOND),
              TUMBLE_END(ts, INTERVAL {{ .Values.flink.attributeInsertWindow |squote }} SECOND)
            from attributes_insert
            group by `id`, `datasetId`, TUMBLE(ts, INTERVAL {{ .Values.flink.attributeInsertWindow |squote }} SECOND));

In [ ]:
%flink_sql_replace -s '{{{{.Values.flink.alertWindow | squote}}}}' -r "'0.5'"

In [ ]:
%%flink_gateway_sql_query
    
WITH WindowedAlerts AS (
    SELECT 
        `resource`, 
        `event`,
        `environment`,
        `service`,
        `customer`,
        LAST_VALUE(severity) AS `severity`,
        LAST_VALUE(`text`) AS `text`,
        TUMBLE_START(`ts`, INTERVAL {{.Values.flink.alertWindow | squote}} SECOND) AS window_start,
        TUMBLE_END(`ts`, INTERVAL {{.Values.flink.alertWindow | squote}} SECOND) AS window_end,
        TUMBLE_ROWTIME(`ts`, INTERVAL {{.Values.flink.alertWindow | squote}} SECOND) AS window_time
    FROM TABLE(
        TUMBLE(TABLE alerts_bulk, DESCRIPTOR(ts), INTERVAL {{.Values.flink.alertWindow | squote}} SECOND)
    )
    GROUP BY 
        `resource`, 
        `event`, 
        `service`, 
        `customer`, 
        `environment`, 
        TUMBLE(`ts`, INTERVAL {{.Values.flink.alertWindow | squote}} SECOND)
),
AlertsWithPreviousSeverity AS (
    SELECT
        `resource`,
        `event`,
        `environment`,
        `service`,
        `customer`,
        `severity`,
        `text`,
        `window_end`,
        `window_time`,
        LAG(`severity`) OVER (
            PARTITION BY `resource`, `event`, `service`, `customer`, `environment`
            ORDER BY `window_time`
        ) AS prev_severity
    FROM WindowedAlerts
)
SELECT 
    `resource`,
    `event`,
    `environment`,
    `service`,
    `severity`,
    `customer`,
    `text`
FROM 
    AlertsWithPreviousSeverity
WHERE 
    prev_severity IS NULL OR severity <> prev_severity;


